In [1]:
import numpy as np
caffe_root = '../../'
import sys
sys.path.insert(0, caffe_root+'python')
import caffe

caffe.set_mode_gpu()
net = caffe.Net('deploy.prototxt','../nin/nin_iter_18000.caffemodel', caffe.TEST)

In [23]:
def getImage(loc):
    from PIL import Image
    img = Image.open(loc)
    
    return np.array(img)

def predict(net, inputs_pre, oversample=True):
    transformer = caffe.io.Transformer({'data': (20,3,256,256)})
    transformer.set_transpose('data', (2,0,1))
    #transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
    #transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB
    transformer.set_mean('data',np.load('./imagenet_mean.npy')) # mean pixel
    
    inputs = []
    for in_ in inputs_pre:
        # Take center crop.
        center = np.array([256,256]) / 2.0
        crop = np.tile(center, (1, 2))[0] + np.concatenate([
            np.array([-224,-224]) / 2.0,
            np.array([224,224]) / 2.0
        ])
        inputs.append( transformer.preprocess('data',np.array(in_,np.float))[:, crop[0]:crop[2], crop[1]:crop[3]] )
    out = net.forward_all(data=np.array(inputs))
    predictions = out['pool4']

    return predictions

In [24]:
loc_labels = open('/storage/datasets/mit-places/trainvalsplit_places205/val_places205.txt').readlines()

locs = [];
for i in range(0,205):
    locs.append([])
    
for loc_label in loc_labels:
    loc,label = loc_label.split(' ')
    locs[int(label)].append(loc)
    
labels = [];
for i in range(0,205):
    labels.append(('_').join(locs[i][0].split('/')[7:-1]))

In [ ]:
from IPython.display import display, clear_output
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time

cnt = 0
correct = 0
top5_correct=0

for label,loc_list in enumerate(locs):
    for loc in loc_list :
        img = getImage(loc)
        
        if( img.shape != (256,256,3) ) :
#            continue;
            temp = np.zeros((256,256,3),np.uint8)
            temp[:,:,0] = img
            temp[:,:,1] = img
            temp[:,:,2] = img
            img = temp;
            #plt.imshow(img)
            #plt.show()
            
        prediction = predict(net,[img],oversample=False).reshape(205)
        
        if prediction.argmax() == label :
            correct = correct + 1
        top_5 = prediction.argsort()[-5:][::-1]
        if label in top_5 :
            top5_correct = top5_correct + 1
        cnt = cnt + 1
    
    import sys;
    print "Middle Result : Top-1(%.2f), Top-5(%.2f)"%(float(correct)/float(cnt)*100, float(top5_correct)/float(cnt)*100)
    sys.stdout.flush() 
print "Result : Top-1(%.2f), Top-5(%.2f)"%(float(correct)/float(cnt), float(top5_correct)/float(cnt))
        
#         print loc, labels[prediction.argmax()], labels[label]

#         raw_input()
#         clear_output()

In [60]:
test_locs = open('/storage/datasets/mit-places/imgList.csv').readlines()
f = file('result_nin.txt', 'w')

cnt = 0
for loc in test_locs :
    img = getImage(loc.strip())
    if( img.shape != (256,256,3) ) :
        temp = np.zeros((256,256,3),np.uint8)
        temp[:,:,0] = img
        temp[:,:,1] = img
        temp[:,:,2] = img
        img = temp;
#     plt.imshow(img); plt.show();

    prediction = predict(net,[img],oversample=False).reshape(205)
    out = "%s\n"%(', '.join(str(x) for x in prediction.argsort()[-5:][::-1]))
    f.write(out)
    
    cnt = cnt + 1
    if( cnt % 205 == 0 ):
        print "%f percent Done."%(float(cnt)/(205*2))

f.close()
#     print loc
#     for i in prediction.argsort()[-5:][::-1]:
#         print labels[i]
#     break;


0.005000 percent Done.
0.010000 percent Done.
0.015000 percent Done.
0.020000 percent Done.
0.025000 percent Done.
0.030000 percent Done.
0.035000 percent Done.
0.040000 percent Done.
0.045000 percent Done.
0.050000 percent Done.
0.055000 percent Done.
0.060000 percent Done.
0.065000 percent Done.
0.070000 percent Done.
0.075000 percent Done.
0.080000 percent Done.
0.085000 percent Done.
0.090000 percent Done.
0.095000 percent Done.
0.100000 percent Done.
0.105000 percent Done.
0.110000 percent Done.
0.115000 percent Done.
0.120000 percent Done.
0.125000 percent Done.
0.130000 percent Done.
0.135000 percent Done.
0.140000 percent Done.
0.145000 percent Done.
0.150000 percent Done.
0.155000 percent Done.
0.160000 percent Done.
0.165000 percent Done.
0.170000 percent Done.
0.175000 percent Done.
0.180000 percent Done.
0.185000 percent Done.
0.190000 percent Done.
0.195000 percent Done.
0.200000 percent Done.
0.205000 percent Done.
0.210000 percent Done.
0.215000 percent Done.
0.220000 pe

In [56]:
# from os import listdir
# from os.path import isfile, join

# test_locs = open('/storage/datasets/mit-places/imgList.csv').readlines()
# onlyfiles = [ '/storage/datasets/mit-places/testSet_resize/c'+f[1:] for f in listdir('/storage/datasets/mit-places/testSet_resize/') if isfile(join('/storage/datasets/mit-places/testSet_resize/',f)) ]
# correctfiles = [ '/storage/datasets/mit-places/testSet_resize/'+f for f in listdir('/storage/datasets/mit-places/testSet_resize/') if isfile(join('/storage/datasets/mit-places/testSet_resize/',f)) ]

In [57]:
# f = file('imgList.txt', 'w')
# test_locs = open('/storage/datasets/mit-places/imgList.csv').readlines()
# for loc in test_locs :
#     out = '%s\n'%correctfiles[onlyfiles.index(loc.strip())]
#     f.write(out)
# f.close()  
#     #print [item for item, count in collections.Counter(onlyfiles).items() if count > 1]